In [70]:
#gee URLError: <urlopen error [WinError 10061] 由于目标计算机积极
# https://blog.csdn.net/q2121213/article/details/120827660

import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'  
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

import ee
ee.Authenticate()

ee.Initialize()

Enter verification code: 4/1AfJohXlfnY73v41NwYYfOLhLPg7TPEEUZh99HjOrRTA6dM3asq4Mk596Lvw

Successfully saved authorization token.


In [77]:

# # Wa
# grid = ee.FeatureCollection("projects/ee-xianglingliwei/assets/Washington_grid")
# state = "Wa";

# Ca
grid = ee.FeatureCollection("projects/ee-xianglingliwei/assets/California_grid")
state = "Ca"

# # Xinjiang
# grid = ee.FeatureCollection("projects/ee-xianglingliwei/assets/Xinjiang_grid")
# state = "XJ"

# # Wa
# year = 2020
# month = 7
# month = 8
# month = 9
# month = 10

# Ca
year = 2019
# month = 7
# month = 8
# month = 9
month = 10

if month in [1,3,5,7,8]:   
    start_date=ee.Date('{}-0{}-01'.format(year, month))
    end_date=ee.Date('{}-0{}-31'.format(year, month))
elif month in [10,12]:   
    start_date=ee.Date('{}-{}-01'.format(year, month)) 
    end_date=ee.Date('{}-{}-31'.format(year, month))
elif month in [4,6,9]:   
    start_date=ee.Date('{}-0{}-01'.format(year, month))
    end_date=ee.Date('{}-0{}-30'.format(year, month))
elif month in [11]:   
    start_date=ee.Date('{}-{}-01'.format(year, month))
    end_date=ee.Date('{}-{}-30'.format(year, month))


cloud_percent = 20

# # Wa
# saveDir = 'Wa_grid_Sentinel'

# Ca
saveDir = 'Ca_grid_Sentinel'

# 保存影像的坐标  
# # Wa
# crs = 'EPSG:32611'
# EPSG:4326 - WGS 84
# crs = 'EPSG:4326'

# # Ca
# crs = 'EPSG:32611'
# # EPSG:4326 - WGS 84
crs = 'EPSG:4326'

sentinel_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                       .filterDate(start_date, end_date)
                       .filterBounds(grid)
                       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percent)))

In [78]:

def download_image(feature):
    feature = ee.Feature(feature)
#     image_id = ee.String(feature.get('id')).getInfo() 
#     image_id = str(feature.get('id').getInfo()) 
    image_id = feature.get('id').getInfo() 
    if image_id < 10:
        name_id = "00" + str(image_id) 
    elif 10 <= image_id < 100:
        name_id = "0" + str(image_id)
    else:
        name_id = str(image_id)
     
#     print(type(image_id))
    geometry = feature.geometry()
    
    image = (sentinel_collection
             .filterBounds(geometry)
             .select(['B2','B3','B4','B8'])
             .median()  
             .clip(geometry))  
    
    print("{0}月---------id_{1}".format(month, name_id))   
#     month = "{}".format(month)
    
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=state + "_" +  "{}".format(month) +  "_" + name_id,  
        fileNamePrefix=state + "_" + "val" + "_" + "{}".format(month) +  "_" + name_id,
        folder=saveDir,  
        scale=10,  
        region=geometry,
        maxPixels=1e13,
        crs=crs
    )
    task.start()

In [81]:

to_download_grid = grid.filter(ee.Filter.eq('irrigation', 'T'))

to_download_list = to_download_grid.getInfo()['features']

i = 1
for feature in to_download_list:
    print("第{}个".format(i))
    feature = ee.Feature(feature)
    image_id = feature.get('id').getInfo()
    if image_id in [1,12,23,34,106,107,108,109,93,94,95,96,97,98,81,82,83,84,85,87,69,70,71,72,73,75,58,59,60,61,62,46,47,
                   48,49,50,51,35,36,37,38,39,24,25]:
        continue
    download_image(feature)
    i = i + 1

第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
第1个
10月---------id_026
第2个
10月---------id_027
第3个
第3个
10月---------id_013
第4个
10月---------id_014
第5个
10月---------id_015
第6个
10月---------id_016
第7个
第7个
10月---------id_002
第8个
10月---------id_003
第9个
10月---------id_004
第10个
10月---------id_005
